## Homework: Agents

In this homework, we will learn more about function calling,
and we will also explore MCP - model-context protocol. 


## Preparation

First, we'll define a function that we will use when building
our agent. 

It will generate fake weather data:


In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

## Q1. Define function description

We want to use it as a tool for our agent, so we need to 
describe it 

How should the description for this function look like? Fill in missing parts


get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Retrieve the temperature for a specific city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city to get weather data for"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

Answers:

- TODO1 → "get_weather"  ← matches the function name
- TODO2 → "Retrieve the temperature for a specific city"
- TODO3 → "city"
- TODO4 → "Name of the city to get weather data for"
- TODO5 → "city" (as a string in a list: ["city"])

## Testing it (Optional)

If you have OpenAI API Key (or alternative provider),
let's test it.

A question could be "What's the weather like in Germany?"

Experiment with different system prompts to have better answers
from the system.

You can use [chat_assistant.py](https://github.com/alexeygrigorev/rag-agents-workshop/blob/main/chat_assistant.py)
or implement everything yourself 

## Q2. Adding another tool

Let's add another tool - a function that can add weather data
to our database:

In [2]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

Now let's write a description for it.

What did you write?

Optionally, you can test it after adding this function.

In [3]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set the temperature for a specific city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city to set weather data for"
            },
            "temp": {
                "type": "number",
                "description": "Temperature to associate with the city"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

## MCP

MCP stands for Model-Context Protocol. It allows LLMs communicate
with different tools (like Qdrant). It's function calling, but 
one step further:

* A tool can export a list of functions it has
* When we include the tool to our Agent, we just need to include the link to the MCP server

## Q3. Install FastMCP

Let's install a library for MCP - [FastMCP](https://github.com/jlowin/fastmcp):

```bash
pip install fastmcp
```

What's the version of FastMCP you installed?

In [4]:
!pip list |grep fastmcp

fastmcp                   2.10.5


## Q4. Simple MCP Server 

A simple MCP server from the documentation looks like that:

In our case, we need to write docstrings for our functions.

Let's ask ChatGPT for help:

In [5]:
!python weather_server.py



╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Weather Server ☁️                                    │
│    📦 Transport:       STDIO                                               │
│                                                

So, for 'TODO' the answer is:

'stdio'

## Q5. Protocol

There are different ways to communicate with an MCP server.
Ours is currently running using standart input/output, which
means that the client write something to stdin and read the
answer using stdout.

Our weather server is currently running.

This is how we start communitcating with it:

- First, we send an initialization request -- this way, we register our client with the server:

In [6]:
# json_rpc_test.py
import subprocess
import json
import time

# Start the server process
process = subprocess.Popen(
    ["python", "weather_server.py"],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

def send_msg(obj):
    json_str = json.dumps(obj) + '\n'
    process.stdin.write(json_str)
    process.stdin.flush()
    print(">> Sent:", json_str.strip())


def read_response():
    line = process.stdout.readline()
    if line:
        print("<< Received:", line.strip())


In [7]:
# Step 1: Initialize
send_msg({
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "protocolVersion": "2024-11-05",
        "capabilities": {
            "roots": {"listChanged": True},
            "sampling": {}
        },
        "clientInfo": {
            "name": "test-client",
            "version": "1.0.0"
        }
    }
})
read_response()

>> Sent: {"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
<< Received: {"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Weather Server ☁️","version":"1.12.0"}}}


In [8]:
# Step 2: Initialized
send_msg({"jsonrpc": "2.0", "method": "notifications/initialized"})
time.sleep(0.1)  # Give server time

# Step 3: List tools
send_msg({"jsonrpc": "2.0", "id": 2, "method": "tools/list"})
read_response()

>> Sent: {"jsonrpc": "2.0", "method": "notifications/initialized"}
>> Sent: {"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
<< Received: {"jsonrpc":"2.0","id":2,"result":{"tools":[{"name":"get_weather","description":"Retrieve the temperature for a specific city.","inputSchema":{"properties":{"city":{"title":"City","type":"string"}},"required":["city"],"type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"number"}},"required":["result"],"title":"_WrappedResult","type":"object","x-fastmcp-wrap-result":true}},{"name":"set_weather","description":"Set the temperature for a specific city.","inputSchema":{"properties":{"city":{"title":"City","type":"string"},"temp":{"title":"Temp","type":"number"}},"required":["city","temp"],"type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"string"}},"required":["result"],"title":"_WrappedResult","type":"object","x-fastmcp-wrap-result":true}}]}}


In [9]:
# Step 4: Call get_weather
send_msg({
    "jsonrpc": "2.0",
    "id": 3,
    "method": "tools/call",
    "params": {
        "name": "get_weather",
        "arguments": {"city": "Berlin"}
    }
})
read_response()

>> Sent: {"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "Berlin"}}}
<< Received: {"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}


- What did you get in response?

20

## Q6. Client

We typically don't interact with the server by copy-pasting 
commands in the terminal.

In practice, we use an MCP Client. Let's implement it. 

FastMCP also supports MCP clients:

Use the client to get the list of available tools
of our script. How does the result look like?

In [10]:
from fastmcp import Client

import weather_server
async with Client(weather_server.mcp) as mcp_client:
    tools = await mcp_client.list_tools()
    print("Available tools:")
    for tool in tools:
        print(tool)

Available tools:
name='get_weather' title=None description='Retrieve the temperature for a specific city.' inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True} annotations=None meta=None
name='set_weather' title=None description='Set the temperature for a specific city.' inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 'type': 'number'}}, 'required': ['city', 'temp'], 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True} annotations=None meta=None


## Using tools from the MCP server (optional)

FastMCP uses asyncio for client-server communication. In our
case, the code we wrote previously in the module
(chat_assistant.py) is not asyncio-friendly, so it will
require a lot of adjustments to run it. 

Which is why we asked Claude to implement a simple
non-async MCP client (see [mcp_client.py](mcp_client.py))
that can only do this:

- List tools
- Invoke the specified tool

Note: this is not a production-ready MCP Client! Use it
only for learning purposes.

Check the code - it's quite illustrative. Or experiment
with writing this code yourself.

Here's how we can use it:

In [11]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "weather_server.py"])
our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

In [12]:
import json

class MCPTools:
    def __init__(self, mcp_client):
        self.mcp_client = mcp_client
        self.tools = None

    def get_tools(self):
        if self.tools is None:
            mcp_response = self.mcp_client.get_tools()

            if isinstance(mcp_response, dict) and "tools" in mcp_response:
                mcp_tools = mcp_response["tools"]
            else:
                raise ValueError("Unexpected response format from MCP client")

            self.tools = convert_tools_list(mcp_tools)

        return self.tools

    def function_call(self, tool_call_response):
        function_name = tool_call_response["name"]
        arguments = json.loads(tool_call_response["arguments"])
        result = self.mcp_client.call_tool(function_name, arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response["call_id"],
            "output": json.dumps(result, indent=2),
        }

def convert_tools_list(tools):
    return [
        {
            "type": "function",
            "function": {
                "name": tool["name"],
                "description": tool["description"],
                "parameters": tool["inputSchema"]
            }
        }
        for tool in tools
    ]

In [13]:
developer_prompt = """
You help users find out the weather in their cities.
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

In [14]:
import os
from openai import OpenAI

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.github.ai/inference"
model_name = "openai/gpt-4o"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

# #test client

# response = client.chat.completions.create(
#     messages=[
#         {
#             "role": "system",
#             "content": "You are a helpful assistant.",
#         },
#         {
#             "role": "user",
#             "content": "What is the capital of France?",
#         }
#     ],
#     temperature=1.0,
#     top_p=1.0,
#     max_tokens=1000,
#     model=model_name
# )

# print(response.choices[0].message.content)

In [15]:
import chat_assistant

chat = chat_assistant.ChatAssistant(
    tools=MCPTools(mcp_client=our_mcp_client),
    developer_prompt=developer_prompt,
    chat_interface=chat_assistant.ChatInterface(),
    client=client
)

chat.run()

RateLimitError: Error code: 429 - {'error': {'code': 'RateLimitReached', 'message': 'Rate limit of 50 per 86400s exceeded for UserByModelByDay. Please wait 43414 seconds before retrying.', 'details': 'Rate limit of 50 per 86400s exceeded for UserByModelByDay. Please wait 43414 seconds before retrying.'}}